In [3]:
import defer

In [4]:
operation = defer.Deferred()


In [5]:
def x(result):
    print("Hooray, a result:" + repr(x))


In [6]:
operation.add_callbacks(x)
# ...


In [7]:
def operationDone():
    operation.callback("completed")

In [8]:
def callback(previous, counter=False):
...     if counter:
...         return previous + 1
...     return previous
>>> deferred = Deferred()
>>> deferred.add_callback(callback, counter=True)
>>> deferred.callback(1)
>>> deferred.result
2

NameError: name 'Deferred' is not defined

In [9]:
def callback(previous, counter=False):
    if counter:
        return previous + 1
    return previous

In [10]:
deferred = defer.Deferred()


In [11]:
deferred.add_callback(callback, counter=True)

In [12]:
deferred.callback(4)

In [13]:
deferred.result

5

In [14]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers/sbt_ibroker_accessor"
]

In [15]:
sys.path += paths

In [16]:
import warnings
import pandas as pd
import time
from ibmaster import IBWrapper, contract
from ib.ext.EClientSocket import EClientSocket
import math
from random import randint
from logging import getLogger

import pandas as pd
import defer


warnings.filterwarnings('ignore')


GTC = 'GTC'
IOC = 'IOC'
MKT = 'MKT'
MOO = 'MOO'
MOC = 'MOC'

_log = getLogger('IBRK')

def check_completion(result , default=0):
    try:
        if default != 0:
            len_result = len(result)
            if len_result > 0:
                return result
        elif result:
            return result
    except:
        pass

class IbConnect(object):

    _call_back = None

    def __init__(self, host, port=7497):
        self._tws = None
        self.sys_manager = {}
        self.TICKER_ID = randint(0, 1000000000)
        self.current_order_id = None
        self.order_time = None
        self.host = host
        self.port = port

    def setup_new_connection(self):
        _log.info("Setting up connection to IB")
        self._call_back = IBWrapper()  # Instantiate IBWrapper. callback
        self._tws = EClientSocket(self._call_back)  # Instantiate EClientSocket and return data to callback
        self._tws.eConnect(self.host, self.port, self.client_id)  # Connect to TWS
        self._call_back.initiate_variables()
        self._tws.reqIds(1)

    @property
    def is_connected(self):
        return self._tws is not None and self._tws.isConnected()

    def __connect__(self):
        for retry in range(10, -1, -1):
            try:
                self.setup_new_connection()
                if self._tws and self._tws.isConnected():
                    _log.info("IB connected")
                    break
            except Exception as error:
                if retry == 0:
                    raise
                warnings('Could not connect to Database. Will retry in 1 seconds. {} retries left'.format(retry))
                time.sleep(0.5)
        _log.info("IB Connection succeeded")

    @property
    def client_id(self):
        return randint(0, 50000)

    @property
    def tws(self):
        if self._tws and self._tws.isConnected():
            return self._tws
        self.setup_new_connection()
        return self._tws

    def disconnect(self):
        self.tws.eDisconnect()

    @staticmethod
    def get_defer_result(defer_value):
        while True:
            try:
                rv = defer_value.result
                break
            except:
                time.sleep(0.05)
                continue
        return rv

    @defer.inline_callbacks
    def retrieve_positions(self, account_id=None):
        """Get the current positions from TWS
        Calling this twice without first resolving the callback will overwrite the previous retrieve_positions.
        This causes the other retrieve_positions to stall forever.
        """
        _log.info("Retrieve position")
        if self._call_back.deferred_positions is not None:
            raise RuntimeError("Calling retrieve_positions concurrently is not supported")
        # We set up the deferred positions that the results will be in
        self._call_back.deferred_positions = defer.Deferred()
        # Set this up before calling TWS reqPositions so positionEnd will resolve this deferred
        # Make a request to TWS API to get the current positions
        self.tws.reqPositions()

        position_data = yield self._call_back.deferred_positions
        # We need to clean up the positions
        self._call_back.deferred_positions = None
        self._call_back.update_Position = []
        print(position_data, type(position_data))
        df_positions = pd.DataFrame(position_data,
                                    columns=['Account', 'ContractID', 'Currency', 'Exchange', 'Expiry',
                                             'IncludeExpired', 'LocalSymbol', 'Multiplier', 'Right',
                                             'SecurityType', 'Strike', 'Symbol', 'TradingClass',
                                             'Position', 'AverageCost'])
        if account_id:
            df_positions = df_positions[df_positions["Account"] == account_id.strip()]
        defer.return_value(df_positions)

    @defer.inline_callbacks
    def retrieve_account(self):
        _log.info("Retrieve position")
        if self._call_back.deferred_account is not None:
            raise RuntimeError("Calling retrieve_positions concurrently is not supported")
        # We set up the deferred account that the results will be in
        self._call_back.deferred_account = defer.Deferred()
        # Set this up before calling TWS reqPositions so positionEnd will resolve this deferred
        # Make a request to TWS API to get the current positions
        self.tws.reqAccountSummary(reqId=3, group="All", tags="NetLiquidation")

        account_data = yield self._call_back.deferred_account
        # We need to clean up the positions
        self._call_back.deferred_positions = None
        self._call_back.update_Position = []

        df_account = pd.DataFrame(account_data,
                                  columns=['requestID', 'accountIdKey','tag','value','currency'])

        defer.return_value(df_account)

In [18]:

    # run = IbConnect(port=7497, host='34.238.115.7', account_id='DU2411434')
run = IbConnect(port=8888, host='34.238.115.7')

run.setup_new_connection()



Server Version: 76
TWS Time at connection:20200625 12:57:26 GMT
[-1, 2104, 'Market data farm connection is OK:hfarm']
[-1, 2104, 'Market data farm connection is OK:usfarm.nj']
[-1, 2104, 'Market data farm connection is OK:eufarm']
[-1, 2104, 'Market data farm connection is OK:jfarm']
[-1, 2104, 'Market data farm connection is OK:usfuture']
[-1, 2104, 'Market data farm connection is OK:cashfarm']
[-1, 2104, 'Market data farm connection is OK:usopt']
[-1, 2104, 'Market data farm connection is OK:usfarm']
[-1, 2106, 'HMDS data farm connection is OK:euhmds']
[-1, 2106, 'HMDS data farm connection is OK:fundfarm']
[-1, 2106, 'HMDS data farm connection is OK:ushmds']
[-1, 2158, 'Sec-def data farm connection is OK:secdefnj']


In [19]:
data = run.retrieve_positions()


[('DU1447869', 756733, 'USD', 'ARCA', None, False, 'SPY', None, None, 'STK', 0.0, 'SPY', 'SPY', 50, 313.51), ('DU1447869', 4391, 'USD', 'NASDAQ', None, False, 'AMD', None, None, 'STK', 0.0, 'AMD', 'NMS', 100, 54.19), ('DU1447869', 400811583, 'USD', None, '20200717', False, 'SPY   200717P00310000', '100', 'P', 'OPT', 310.0, 'SPY', 'SPY', 20, 869.6888)] <class 'list'>


In [20]:
data.result

Account  ContractID Currency Exchange    Expiry  IncludeExpired  \
0  DU1447869      756733      USD     ARCA      None           False   
1  DU1447869        4391      USD   NASDAQ      None           False   
2  DU1447869   400811583      USD     None  20200717           False   

             LocalSymbol Multiplier Right SecurityType  Strike Symbol  \
0                    SPY       None  None          STK     0.0    SPY   
1                    AMD       None  None          STK     0.0    AMD   
2  SPY   200717P00310000        100     P          OPT   310.0    SPY   

  TradingClass  Position  AverageCost  
0          SPY        50     313.5100  
1          NMS       100      54.1900  
2          SPY        20     869.6888

[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.hfarm']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.eufarm']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.jfarm']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usfuture']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.hfarm']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.usfuture']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.eufarm']
[-1, 2108, 'Market data farm connection is inactive but should be available upon demand.jfarm']


In [31]:
url = "https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/{Symbol}~{Exchange}"
for  row in data.result.iterrows():
    print(url.format(**row[1]))

https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/ESGV~BATS
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/ESGD~NASDAQ
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/AMD~NASDAQ
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/DVY~NASDAQ
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/VO~ARCA
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/VDE~ARCA
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/TFI~ARCA
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/AAPL~None
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/SPY~ARCA
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/IBKR~NASDAQ
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/TIP~ARCA
https://dev-api.stansberryterminal.com/api/timeseries/market/cached/quote/BW

In [35]:
for row in data.result.iterrows():
    
    try:
        price = run.retrieve_market_data_ib(row[1])
        price = run.get_defer_result(price)
    except:
        continue
    print(price)
    

[]
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute]
Index: []
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute, Type]
Index: []
[]
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute]
Index: []
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute, Type]
Index: []
[]
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute]
Index: []
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute, Type]
Index: []
[]
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute]
Index: []
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute, Type]
Index: []
[]
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute]
Index: []
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute, Type]
Index: []
[]
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute]
Index: []
Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute, Type]
Index: []
[]
Empty DataFrame
Columns: [Ticke

In [108]:
price.result

Empty DataFrame
Columns: [TickerId, Field, Price, CanAutoExecute, Type]
Index: []

In [44]:

def get_result(defer_value):
    while True:
        try:
            rv = data.result
            break
        except:
            time.sleep(0.05)
            continue
    return rv

In [44]:
a = "AAPL 200918P00390000".replace(" ","")
a[0:-9]
a[-9:]

'P00390000'

In [43]:
a = "20200918"
a[0:6]

'202009'

In [ ]:
data = run.retrieve_positions()
result = get_result(data)
result

In [38]:
data = run.retrieve_positions()
result = get_result(data)
result

Account  Contract ID Currency Exchange    Expiry  Include Expired  \
0    DUC00074    333970434      USD     BATS      None            False   
1    DUC00074    238760476      USD   NASDAQ      None            False   
2   DU2411434         4391      USD   NASDAQ      None            False   
3    DUC00074     26653236      USD   NASDAQ      None            False   
4    DUC00074     27638087      USD     ARCA      None            False   
5    DUC00074     27684036      USD     ARCA      None            False   
6    DUC00074    225020107      USD     ARCA      None            False   
7   DU2411434    395900257      USD     None  20200918            False   
8   DU2411434       756733      USD     ARCA      None            False   
9    DUC00074     43645865      USD   NASDAQ      None            False   
10   DUC00074     26787475      USD     ARCA      None            False   
11   DUC00074     46577293      USD     ARCA      None            False   
12   DUC00074     15547816      USD     ARCA      None            False   
13   DUC00074    313845618      USD     BATS      None            False   
14   DUC00074     31230302      USD     ARCA      None            False   
15   DUC00074    238760480      USD   NASDAQ      None            False   
16   DUC00074    338719585      USD     ARCA      None            False   

             Local Symbol Multiplier Right Security Type  Strike Symbol  \
0                    ESGV       None  None           STK     0.0   ESGV   
1                    ESGD       None  None           STK     0.0   ESGD   
2                     AMD       None  None           STK     0.0    AMD   
3                     DVY       None  None           STK     0.0    DVY   
4                      VO       None  None           STK     0.0     VO   
5                     VDE       None  None           STK     0.0    VDE   
6                     TFI       None  None           STK     0.0    TFI   
7   AAPL  200918P00390000        100     P           OPT   390.0   AAPL   
8                     SPY       None  None           STK     0.0    SPY   
9                    IBKR       None  None           STK     0.0   IBKR   
10                    TIP       None  None           STK     0.0    TIP   
11                    BWX       None  None           STK     0.0    BWX   
12                    LQD       None  None           STK     0.0    LQD   
13                   ESML       None  None           STK     0.0   ESML   
14                    VNQ       None  None           STK     0.0    VNQ   
15                   ESGE       None  None           STK     0.0   ESGE   
16                   EAGG       None  None           STK     0.0   EAGG   

   Trading Class  Position  Average Cost  
0           ESGV      1142     52.776556  
1            NMS       954     64.171528  
2            NMS       100     54.770000  
3            NMS       292    101.061767  
4             VO        50    170.488810  
5            VDE        99     81.962902  
6            TFI      1109     50.506440  
7           AAPL         1   4218.503800  
8            SPY       100    310.630000  
9            NMS      2800     39.405838  
10           TIP       207    115.004653  
11           BWX       712     28.577252  
12           LQD        99    124.438912  
13          ESML       309     27.386963  
14           VNQ       137     88.732232  
15           NMS      1221     33.994677  
16          EAGG      1280     53.302290

In [ ]:
data.

In [20]:
data.result

Account  Contract ID Currency Exchange Expiry  Include Expired  \
0   DUC00074    333970434      USD     BATS   None            False   
1   DUC00074    238760476      USD   NASDAQ   None            False   
2   DUC00074     26653236      USD   NASDAQ   None            False   
3   DUC00074     27638087      USD     ARCA   None            False   
4   DUC00074     27684036      USD     ARCA   None            False   
5   DUC00074    225020107      USD     ARCA   None            False   
6   DUC00074     43645865      USD   NASDAQ   None            False   
7   DUC00074     26787475      USD     ARCA   None            False   
8   DUC00074     46577293      USD     ARCA   None            False   
9   DUC00074     15547816      USD     ARCA   None            False   
10  DUC00074    313845618      USD     BATS   None            False   
11  DUC00074     31230302      USD     ARCA   None            False   
12  DUC00074    238760480      USD   NASDAQ   None            False   
13  DUC00074    338719585      USD     ARCA   None            False   

   Local Symbol Multiplier Right Security Type  Strike Symbol Trading Class  \
0          ESGV       None  None           STK     0.0   ESGV          ESGV   
1          ESGD       None  None           STK     0.0   ESGD           NMS   
2           DVY       None  None           STK     0.0    DVY           NMS   
3            VO       None  None           STK     0.0     VO            VO   
4           VDE       None  None           STK     0.0    VDE           VDE   
5           TFI       None  None           STK     0.0    TFI           TFI   
6          IBKR       None  None           STK     0.0   IBKR           NMS   
7           TIP       None  None           STK     0.0    TIP           TIP   
8           BWX       None  None           STK     0.0    BWX           BWX   
9           LQD       None  None           STK     0.0    LQD           LQD   
10         ESML       None  None           STK     0.0   ESML          ESML   
11          VNQ       None  None           STK     0.0    VNQ           VNQ   
12         ESGE       None  None           STK     0.0   ESGE           NMS   
13         EAGG       None  None           STK     0.0   EAGG          EAGG   

    Position  Average Cost  
0       1142     52.776556  
1        954     64.171528  
2        292    101.061767  
3         50    170.488810  
4         99     81.962902  
5       1109     50.506440  
6       2800     39.405838  
7        207    115.004653  
8        712     28.577252  
9         99    124.438912  
10       309     27.386963  
11       137     88.732232  
12      1221     33.994677  
13      1280     53.302290

In [21]:
test = data.result

In [24]:
for row in test.iterrows():
    row[1]["current_price"] = 1
    print(row[0], row[1][["Position", "Symbol", "current_price"]])
#     print (row[0], row[1]["Symbol", "Position"])

0 Position         1142
Symbol           ESGV
current_price       1
Name: 0, dtype: object
1 Position          954
Symbol           ESGD
current_price       1
Name: 1, dtype: object
2 Position         292
Symbol           DVY
current_price      1
Name: 2, dtype: object
3 Position         50
Symbol           VO
current_price     1
Name: 3, dtype: object
4 Position          99
Symbol           VDE
current_price      1
Name: 4, dtype: object
5 Position         1109
Symbol            TFI
current_price       1
Name: 5, dtype: object
6 Position         2800
Symbol           IBKR
current_price       1
Name: 6, dtype: object
7 Position         207
Symbol           TIP
current_price      1
Name: 7, dtype: object
8 Position         712
Symbol           BWX
current_price      1
Name: 8, dtype: object
9 Position          99
Symbol           LQD
current_price      1
Name: 9, dtype: object
10 Position          309
Symbol           ESML
current_price       1
Name: 10, dtype: object
11 Position       

In [60]:
df_positions = pd.DataFrame(data.result,
                                    columns=['Account', 'Contract ID', 'Currency', 'Exchange', 'Expiry',
                                             'Include Expired', 'Local Symbol', 'Multiplier', 'Right',
                                             'Security Type', 'Strike', 'Symbol', 'Trading Class',
                                             'Position', 'Average Cost'])

In [61]:
df_positions

Account  Contract ID Currency Exchange Expiry  Include Expired  \
0  DU1447869       756733      USD     ARCA   None            False   

  Local Symbol Multiplier Right Security Type  Strike Symbol Trading Class  \
0          SPY       None  None           STK     0.0    SPY           SPY   

   Position  Average Cost  
0        50        313.51

In [50]:
defer_ = defer.Deferred()

In [54]:
defer_.callback()

AlreadyCalledDeferred: 

In [53]:
defer_.result